<a href="https://colab.research.google.com/github/ElZho/ElZho/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
with open('/content/drive/MyDrive/ODS/Text.txt', encoding='utf-8') as file:
    contents = file.read()

In [ ]:
 file.close

<function TextIOWrapper.close>

In [ ]:
contents=contents.lower()
text=contents.replace(r'\,', '')
text=contents.replace(r"\’", "")
text=contents.replace(r"\.", "")
text=contents.replace(r"\W", " ")
text

'\ufeffsustainable supermarkets\nmany of the major supermarket chains have come under fire with accusations of various unethical acts over the past decade. they’ve wasted tonnes of food, they’ve underpaid their suppliers and they’ve contributed to excessive plastic waste in their packaging, which has had its impact on our environment.\nbut supermarkets and grocers are starting to sit up and take notice. in response to growing consumer backlash against the huge amounts of plastic waste generated by plastic packaging, some of the largest uk supermarkets have signed up to a pact promising to transform packaging and cut plastic wastage. in a pledge to reuse, recycle or compost all plastic wastage by 2025, supermarkets are now beginning to take some responsibility for the part they play in contributing to the damage to our environment, with one major supermarket announcing their plan to eliminate all plastic packaging in their own-brand products by 2023.\nin response to criticisms over food

In [ ]:

def W_2_v(data):
  import torch
  from torch import nn
  from torch.autograd import Variable
  import torch.nn.functional as F
  # from collections import defaultdict
  import numpy as np  
  import random

  data=data.split()
  len_t=len(data)
  vocabulary = set(data)
  v_size = len(vocabulary)  
  word2idx={w: idx for (idx, w) in enumerate(vocabulary)}
  w_z=2
  l_r=0.01
  epochs=100
  emb_dims=12             
        
  def get_pairs(data, w_z, len_t, word2idx ):
      pairs=[]
      for i1, w1 in enumerate(data): 
        i2_beg = max(0, i1-w_z)
        for i2 in range(i2_beg, min(len_t, i1+w_z+1) ):            
          if i2 != i1: 
            pairs.append((w1, data[i2])) 
      
      random_seed = 342     
      random.shuffle(pairs)
         

      X = np.zeros((len(pairs), 1),   dtype=np.int64)
      Y = np.zeros((len(pairs)),      dtype=np.int64) 
      for i, (u,w) in enumerate(pairs):        
        X[i,0] = word2idx[u]
        Y[i]   = word2idx[w]      
      return X, Y  

      
  def get_weights(v_size, data, word2idx, len_t): 
        idf = np.zeros( (v_size,), dtype=np.float32)            
        for i, w in enumerate(data):                                             
          idf[word2idx[w]] += 1            
        idf = ( 1+np.log(len_t/(idf+1), dtype=np.float32) )
        return idf*(v_size/idf.sum())        

  class SkipGram(nn.Module):
      def __init__(self, V_DIM, E_DIM): 
          super(SkipGram, self).__init__()           
          self.emb1 = nn.Embedding(V_DIM, E_DIM, scale_grad_by_freq=True) 
          self.flat = nn.Flatten()
          self.linear=nn.Linear(E_DIM, V_DIM, bias=False)   
      def forward(self, x):
          out=self.emb1(x)
          out=self.flat(out)
          out=self.linear(out)
          return out
  def train(X, Y, model, optimizer, criterion, epochs):               
      loss_list = np.zeros((epochs,))   

      for epo in range(epochs):        
        for it in range(0, len(X), 16):          
          xb = torch.from_numpy( X[it: it+16] ).to(device)
          yb = torch.from_numpy( Y[it: it+16] ).to(device)
          
          model.train()
          y = model(xb)
          loss = criterion(y, yb)          
          loss_list[epo] += np.sum(loss.detach().cpu().numpy())                
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          # scheduler.step(loss) 
        # loss_list[epo] = loss.item()  
        # loss_list[epo] /= (len(X)/16)              
      #   if epo % 5 == 0:                  
      #         print(f'Loss at epo {epo}: {loss.item()}')              
      # print(f'Loss at epo {epo}: {loss.item()}')  
      E = model.emb1.weight              
      W = model.linear.weight              
      new_E = 0.5*(model.emb1.weight+model.linear.weight)             
      return  new_E #0.5*(model.emb1.weight+model.linear.weight)   
  def make_dict(Emb, word2idx):
    d={}
    for word, val in word2idx.items():
      emb = Emb[val]         
      d.update({word:emb.detach().numpy() })
    return d 
  
  
  if torch.cuda.is_available():     
    device = torch.device("cuda")
  else:    
    device = torch.device("cpu")

  X, Y= get_pairs(data, w_z, len_t, word2idx )
  weights= get_weights(v_size, data, word2idx, len_t)
  # print(l_r, emb_dims, epochs)
  

  model= SkipGram(v_size, emb_dims).to(device) 
      
  optimizer = torch.optim.Adam(model.parameters(), lr=l_r)
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',\
  #                           factor=0.95, threshold=0.1, min_lr=0.01, verbose=True)
          
  criterion = nn.CrossEntropyLoss(torch.tensor(weights)) 
  
  Embeding=train(X, Y, model, optimizer, criterion, epochs)
  
  
  return make_dict(Embeding, word2idx)

In [ ]:
%%time
d1=W_2_v(text)

CPU times: user 12.3 s, sys: 28 ms, total: 12.3 s
Wall time: 12.4 s
